In [1]:
import numpy as np
import pandas as pd

# DATA

In [8]:
file_path = '../../src/data/temp/processed_zbp_detail_data.csv'
zbp_detail = pd.read_csv(file_path)
zbp_detail

,zip,naics,est,n1_4,n5_9,n10_19,n20_49,n50_99,n100_249,n250_499,n500_999,n1000,year
0,91901,23,88,68,7,6,5,1,0,1,0,0,2012
1,91901,31,2,1,1,0,0,0,0,0,0,0,2012
2,91901,42,9,9,0,0,0,0,0,0,0,0,2012
3,91901,44,67,25,21,16,4,1,0,0,0,0,2012
4,91901,48,8,8,0,0,0,0,0,0,0,0,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17179,92182,71,3,0,0,0,0,0,0,0,0,0,2021
17180,92182,72,9,3,0,0,3,0,0,0,0,0,2021
17181,92182,81,3,0,0,0,0,0,0,0,0,0,2021
17182,92186,54,3,0,0,0,0,0,0,0,0,0,2021


In [26]:
file_path = '../../src/data/temp/processed_zbp_totals_data.csv'
zbp_totals = pd.read_csv(file_path)
zbp_totals

,zip,emp_nf,emp,qp1_nf,qp1,ap_nf,ap,est,year
0,91901,H,4141,H,36304,H,174786,391,2012
1,91902,G,2265,G,19111,G,81569,349,2012
2,91903,S,0,G,123,G,491,13,2012
3,91905,G,19,S,0,H,748,9,2012
4,91906,D,0,D,0,D,0,27,2012
...,...,...,...,...,...,...,...,...,...
1700,92191,H,100,H,2068,J,11697,4,2021
1701,92192,J,5,J,53,H,259,4,2021
1702,92193,J,39,J,894,J,3745,4,2021
1703,92195,G,4,J,27,J,216,3,2021


In [10]:
file_path = '../../src/data/temp/processed_hh_income_data.csv'
hh_income_data = pd.read_csv(file_path)
hh_income_data

,zip,median_hh_income,year
0,91901,76496,2012
1,91902,86099,2012
2,91905,70000,2012
3,91906,54135,2012
4,91910,55875,2012
...,...,...,...
963,92130,166789,2021
964,92131,153058,2021
965,92139,80594,2021
966,92154,74705,2021


In [11]:
file_path = '../../src/data/temp/processed_total_pop_data.csv'
total_pop_data = pd.read_csv(file_path)
total_pop_data

,zip,total_population,year
0,91901,17034.0,2012
1,91902,17659.0,2012
2,91905,1088.0,2012
3,91906,3679.0,2012
4,91910,73761.0,2012
...,...,...,...
1071,92154,85024.0,2021
1072,92155,340.0,2021
1073,92161,0.0,2021
1074,92173,29634.0,2021


In [39]:
file_path = '../../src/data/temp/processed_retire_detail_data.csv'
retire_data = pd.read_csv(file_path)
retire_data

,zip,total_population,year,total_retirement,total_midcareer
0,91901,17034.0,2012,2691.0,3452.0
1,91902,17659.0,2012,3485.0,4031.0
2,91905,1088.0,2012,101.0,225.0
3,91906,3679.0,2012,395.0,836.0
4,91910,73761.0,2012,9195.0,19426.0
...,...,...,...,...,...
1071,92154,85024.0,2021,14203.0,20110.0
1072,92155,340.0,2021,0.0,302.0
1073,92161,0.0,2021,0.0,0.0
1074,92173,29634.0,2021,4517.0,7147.0


# Features

* presence of anchor business(es) (eg, measured by the number of large businesses or combination of large business and industry)
* % tech
* % bio
* % restaurant
* % small business
* population
* % population working (labor force %)


# Industry Percentages

In [12]:
zip_est = zbp_detail.groupby(['zip', 'year'])['est'].sum()
zip_naics = zbp_detail.groupby(['zip', 'year', 'naics'])['est'].sum()
naics_percentages = (zip_naics/zip_est).unstack().fillna(value=0).add_prefix('naics_').add_suffix('_pct').reset_index()
naics_percentages

naics,zip,year,naics_11_pct,naics_21_pct,naics_22_pct,naics_23_pct,naics_31_pct,naics_42_pct,naics_44_pct,naics_48_pct,...,naics_53_pct,naics_54_pct,naics_55_pct,naics_56_pct,naics_61_pct,naics_62_pct,naics_71_pct,naics_72_pct,naics_81_pct,naics_99_pct
0,91901,2012,0.000000,0.0,0.000000,0.225064,0.005115,0.023018,0.171355,0.020460,...,0.069054,0.112532,0.0,0.063939,0.015345,0.071611,0.020460,0.074169,0.074169,0.0
1,91901,2013,0.000000,0.0,0.002488,0.208955,0.009950,0.017413,0.181592,0.017413,...,0.057214,0.099502,0.0,0.059701,0.017413,0.074627,0.019900,0.094527,0.087065,0.0
2,91901,2014,0.002551,0.0,0.002551,0.219388,0.007653,0.017857,0.178571,0.012755,...,0.061224,0.094388,0.0,0.071429,0.012755,0.076531,0.015306,0.094388,0.086735,0.0
3,91901,2015,0.000000,0.0,0.002519,0.236776,0.010076,0.012594,0.181360,0.012594,...,0.068010,0.105793,0.0,0.052897,0.015113,0.075567,0.012594,0.093199,0.078086,0.0
4,91901,2016,0.000000,0.0,0.002564,0.243590,0.010256,0.012821,0.164103,0.020513,...,0.064103,0.100000,0.0,0.056410,0.015385,0.076923,0.012821,0.094872,0.076923,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1518,92198,2016,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,...,0.000000,0.142857,0.0,0.142857,0.000000,0.000000,0.142857,0.000000,0.285714,0.0
1519,92198,2017,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.0
1520,92198,2018,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.0
1521,92199,2015,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


# Establishment Size Percentages

In [13]:
est_size_percentages = zbp_detail.groupby(['zip', 'year'])[['n1_4', 'n5_9', 'n10_19', 'n20_49', 'n50_99', 'n100_249', 'n250_499', 'n500_999', 'n1000']].sum()
est_size_percentages = est_size_percentages.apply(lambda ser: ser/zip_est, axis=0).add_suffix('_pct').reset_index()
est_size_percentages

,zip,year,n1_4_pct,n5_9_pct,n10_19_pct,n20_49_pct,n50_99_pct,n100_249_pct,n250_499_pct,n500_999_pct,n1000_pct
0,91901,2012,0.649616,0.166240,0.107417,0.056266,0.010230,0.005115,0.002558,0.000000,0.002558
1,91901,2013,0.649254,0.151741,0.099502,0.074627,0.012438,0.007463,0.000000,0.002488,0.002488
2,91901,2014,0.622449,0.165816,0.125000,0.061224,0.012755,0.010204,0.000000,0.000000,0.002551
3,91901,2015,0.637280,0.158690,0.120907,0.062972,0.012594,0.005038,0.000000,0.000000,0.002519
4,91901,2016,0.592308,0.192308,0.123077,0.064103,0.015385,0.010256,0.000000,0.000000,0.002564
...,...,...,...,...,...,...,...,...,...,...,...
1518,92198,2016,0.857143,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1519,92198,2017,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1520,92198,2018,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1521,92199,2015,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000


# Compile Features

In [53]:
zbp_totals_with_features = zbp_totals.merge(naics_percentages, on=['zip', 'year'])
zbp_totals_with_features = zbp_totals_with_features.merge(est_size_percentages, on=['zip', 'year'])
zbp_totals_with_features = zbp_totals_with_features.merge(hh_income_data, on=['zip', 'year'])
zbp_totals_with_features = zbp_totals_with_features.merge(total_pop_data, on=['zip', 'year'])
zbp_totals_with_features = zbp_totals_with_features.merge(retire_data, on=['zip', 'year'])
# drop zip codes with insufficient observations (<5)
zip_code_by_num_observations = zbp_totals_with_features.groupby('zip')['year'].count().sort_values()
zip_codes_with_insufficient_observations = zip_code_by_num_observations[zip_code_by_num_observations<=5].index
zbp_totals_with_features = zbp_totals_with_features[zbp_totals_with_features['zip'].apply(lambda x: x not in zip_codes_with_insufficient_observations)]
zbp_totals_with_features

,zip,emp_nf,emp,qp1_nf,qp1,ap_nf,ap,est,year,naics_11_pct,...,n50_99_pct,n100_249_pct,n250_499_pct,n500_999_pct,n1000_pct,median_hh_income,total_population_x,total_population_y,total_retirement,total_midcareer
0,91901,H,4141,H,36304,H,174786,391,2012,0.000000,...,0.010230,0.005115,0.002558,0.0,0.002558,76496,17034.0,17034.0,2691.0,3452.0
1,91902,G,2265,G,19111,G,81569,349,2012,0.002865,...,0.017192,0.002865,0.000000,0.0,0.000000,86099,17659.0,17659.0,3485.0,4031.0
2,91905,G,19,S,0,H,748,9,2012,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,70000,1088.0,1088.0,101.0,225.0
3,91906,D,0,D,0,D,0,27,2012,0.000000,...,0.000000,0.000000,0.037037,0.0,0.000000,54135,3679.0,3679.0,395.0,836.0
4,91910,G,19799,G,200767,G,805325,1458,2012,0.000686,...,0.021262,0.013032,0.002743,0.0,0.001372,55875,73761.0,73761.0,9195.0,19426.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
951,92130,G,23095,G,902483,G,3156368,2200,2021,0.003636,...,0.016364,0.006818,0.000000,0.0,0.000000,166789,57197.0,57197.0,8191.0,8492.0
952,92131,G,19670,G,484038,G,1826354,1093,2021,0.000000,...,0.022915,0.013749,0.000000,0.0,0.000000,153058,36070.0,36070.0,6807.0,4140.0
953,92139,G,1046,G,7714,G,33466,154,2021,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,80594,35364.0,35364.0,6121.0,7857.0
954,92154,G,19142,G,217203,G,941639,1765,2021,0.000000,...,0.014196,0.009086,0.000000,0.0,0.000000,74705,85024.0,85024.0,14203.0,20110.0


Index([91948, 91980, 92060, 92055, 91963, 92066], dtype='int64', name='zip')